In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import SimpleITK as sitk
from tqdm import tqdm
import json

In [31]:
path_gt = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/internal_test_crop_0.4.csv"
df_gt = pd.read_csv(path_gt)
df_gt.head()

,seriesuid,coordX,coordY,coordZ,w,h,d,lesion,intersection_art,intersection_vein,avg_distance_art
0,Ts0006.nii.gz,299.5,240.5,119.0,3.2,4.0,2.8,aneurysm,0.758333,0.241667,1.394045
1,Ts0007.nii.gz,399.5,335.5,192.0,4.0,4.8,4.4,aneurysm,0.595833,0.005833,0.678162
2,Ts0008.nii.gz,218.5,241.5,180.5,6.4,8.0,4.8,aneurysm,0.430469,0.000000,-0.654344
3,Ts0009.nii.gz,435.5,260.0,152.5,9.6,10.0,6.4,aneurysm,0.580295,0.165690,0.627371
4,Ts0010.nii.gz,249.0,239.0,139.0,10.8,11.6,10.0,aneurysm,0.526328,0.000000,0.065363


In [32]:
path_vessel_seg = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/internal_test_0.4_crop_vessel_v2"
)
path_vessel_seg_edt = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/internal_test_0.4_crop_vessel_edt_v2"
)


files_vessel_seg = list(path_vessel_seg.glob("*.nii.gz"))
files_vessel_seg_edt = list(path_vessel_seg_edt.glob("*.nii.gz"))
path_metadata = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/internal_test_meta_crop.json"
with open(path_metadata, "r") as f:
    metadata = json.load(f)
metadata

{'Ts0001.nii.gz': {'origin': [-156.1865234375,
   -310.1865234375,
   -96.40000915527344],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0002.nii.gz': {'origin': [-122.759765625,
   -250.759765625,
   -389.1999816894531],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0003.nii.gz': {'origin': [-137.73046875,
   -260.23046875,
   -1190.9000244140625],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0004.nii.gz': {'origin': [-123.2587890625,
   -253.7587890625,
   11.500009536743164],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0005.nii.gz': {'origin': [-176.654296875,
   -346.654296875,
   -693.9000244140625],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0006.nii.gz': {'origin': [-114.775390625, -271.275390625, 161.0],
  'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]},
 'Ts0007.nii.gz': {'origin

In [33]:
intersections = []
intersections_vein = []
distances = []
for case_name in tqdm(df_gt["seriesuid"].unique()):

    df_case = df_gt[df_gt["seriesuid"] == case_name]

    header_seg = sitk.ReadImage(str(path_vessel_seg / f"{case_name}"))
    header_edt = sitk.ReadImage(str(path_vessel_seg_edt / f"{case_name}"))
    edt_array = sitk.GetArrayFromImage(header_edt)
    seg_array = sitk.GetArrayFromImage(header_seg)
    seg_array_vein = (seg_array == 2).astype(np.uint8)
    seg_array_ves = (seg_array == 1).astype(np.uint8)
    spacing = metadata[case_name]["spacing"]
    spacing = np.array(spacing)
    origin = metadata[case_name]["origin"]
    origin = np.array(origin)
    for _, row in df_case.iterrows():
        pred_array = np.zeros_like(seg_array)
        coordZ, coordY, coordX = (
            int(row["coordZ"]),
            int(row["coordY"]),
            int(row["coordX"]),
        )

        h, w, d = row["h"], row["w"], row["d"]

        h = h / spacing[0]
        w = w / spacing[1]
        d = d / spacing[2]

        pred_array[
            max(0, int(coordZ - d // 2)) : min(
                pred_array.shape[0], int(coordZ + d // 2)
            ),
            max(0, int(coordY - h // 2)) : min(
                pred_array.shape[1], int(coordY + h // 2)
            ),
            max(0, int(coordX - w // 2)) : min(
                pred_array.shape[2], int(coordX + w // 2)
            ),
        ] = 1
        # get intersection over minimum
        intersection = np.sum(pred_array * seg_array_ves)

        minimum = np.sum(pred_array)
        intersection_vein = np.sum(pred_array * seg_array_vein)
        intersections_vein.append(intersection_vein / minimum)
        # get the center of the aneurysm predict)
        avg_distance = edt_array[coordZ, coordY, coordX]
        print(coordZ, coordY, coordX, h, w, d, avg_distance)
        intersections.append(intersection / minimum)
        distances.append(avg_distance)

  1%|          | 1/102 [00:10<18:19, 10.89s/it]

119 240 299 10.0 8.0 7.000000000000001 1.7320508


  2%|▏         | 2/102 [00:30<26:54, 16.15s/it]

192 335 399 12.0 10.0 11.0 4.1231055


  3%|▎         | 3/102 [00:43<23:58, 14.53s/it]

...

197 266 400 21.0 15.0 11.0 5.7445626
187 268 362 21.0 22.0 24.0 5.8309517


 99%|█████████▉| 101/102 [18:46<00:12, 12.97s/it]

198 262 320 17.0 17.0 15.0 5.8309517


100%|██████████| 102/102 [18:55<00:00, 11.13s/it]

205 202 269 11.0 17.0 14.0 1.7320508


In [28]:
df_gt["intersection_art"] = intersections
df_gt["intersection_vein"] = intersections_vein
df_gt["avg_distance_art"] = distances
df_gt

,seriesuid,coordX,coordY,coordZ,w,h,d,lesion,intersection_art,intersection_vein,avg_distance_art
0,Ts0006.nii.gz,299.5,240.5,119.0,3.2,4.0,2.8,aneurysm,0.758333,0.241667,1.394045
1,Ts0007.nii.gz,399.5,335.5,192.0,4.0,4.8,4.4,aneurysm,0.595833,0.005833,0.678162
2,Ts0008.nii.gz,218.5,241.5,180.5,6.4,8.0,4.8,aneurysm,0.430469,0.000000,-0.654344
3,Ts0009.nii.gz,435.5,260.0,152.5,9.6,10.0,6.4,aneurysm,0.580295,0.165690,0.627370
4,Ts0010.nii.gz,249.0,239.0,139.0,10.8,11.6,10.0,aneurysm,0.526328,0.000000,0.065363
...,...,...,...,...,...,...,...,...,...,...,...
121,Ts0148.nii.gz,310.0,205.5,135.5,8.4,8.0,6.4,aneurysm,0.479375,0.000000,0.019052
122,Ts0149.nii.gz,400.0,266.0,197.0,6.0,8.4,4.4,aneurysm,0.666429,0.000000,0.917257
123,Ts0151.nii.gz,362.5,268.0,187.5,8.8,8.4,9.6,aneurysm,0.442708,0.010606,-0.748059
124,Ts0151.nii.gz,320.0,262.0,198.0,6.8,6.8,6.0,aneurysm,0.614676,0.000000,0.799423


In [29]:
df_gt.to_csv(
    path_gt,
    index=False,
)